### Extracting Data from pdf to dataframe

In [ ]:
import pandas as pd
import pdfplumber

: 

In [ ]:
purchase_table_path = "/Users/saileshpanda/Desktop/IndiaAI/PDF/Purchase.pdf"
redemption_table_path = "/Users/saileshpanda/Desktop/IndiaAI/PDF/Redemption.pdf"

In [ ]:
flag = 0
with pdfplumber.open(purchase_table_path) as pdf:
    #pages = pdf.pages[1]
    for page in pdf.pages:
        table = page.extract_tables()
        for pos,r in enumerate(table[0]):
            if flag == 0:
                purchase_table = pd.DataFrame(columns=r)
                flag = 1
            else:
                if pos==0:
                    continue
                purchase_table.loc[len(purchase_table)] = r

In [ ]:
purchase_table.tail(4)

In [ ]:
flag = 0
with pdfplumber.open(redemption_table_path) as pdf:
    #pages = pdf.pages[1]
    for page in pdf.pages:
        table = page.extract_tables()
        for pos,r in enumerate(table[0]):
            if flag == 0:
                redemption_table = pd.DataFrame(columns=r)
                flag = 1
            else:
                if pos==0:
                    continue
                redemption_table.loc[len(redemption_table)] = r

In [ ]:
redemption_table.tail(4)

In [ ]:
redemption_table.set_index('Sr No.', inplace=True)
purchase_table.set_index('Sr No.', inplace=True)

In [ ]:
purchase_table.head()

### Doc Embedding

In [ ]:
from sentence_transformers import SentenceTransformer
import json

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
purchase_table['Embedding'] = None
for index, rows in purchase_table.iterrows():
    purchase_table["Embedding"][index] = embedding_model.encode(rows.to_csv())

In [ ]:
redemption_table['Embedding'] = None
for index,rows in redemption_table.iterrows():
    redemption_table["Embedding"][index] = embedding_model.encode(rows.to_csv())

In [ ]:
purchase_table.to_csv("purchase_table.csv")
redemption_table.to_csv("redemption_table.csv")

#### Similarity

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
purchase_table["Embedding"][3].shape

In [ ]:
embedding_model.encode(purchase_table.iloc[0, :-1].to_csv()).shape

In [ ]:
similarity = cosine_similarity(purchase_table["Embedding"][14].reshape(1, -2), 
                               embedding_model.encode(" What is the total bond amount of A B C INDIA LIMITED").reshape(1, -1))
print(f'Cosine Similarity: {similarity[0][0]}')

In [ ]:
def find_similar_doc(prompt, purchase_table, redemption_table):
    


### LLM

In [ ]:
import google.generativeai as genai
import gradio as gr
import os

In [ ]:
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel("gemini-pro")

In [ ]:
prompt = purchase_table[:100]
prompt = prompt.to_csv()
prompt = prompt + "         Prompt: Use the above table to return the numbers for yhe below query Query:What is the total bond amount purchased by A B C INDIA LIMITED on 12th April 2019"
#model.generate_content(prompt)

In [ ]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ["GROQ_API_KEY"],
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)